### Handoffs
A way for an agent to invoke another agent

In [1]:
from dotenv import load_dotenv
import os

load_dotenv()

api_key = os.environ.get("OPENAI_API_KEY")
if not api_key:
    raise ValueError("OPENAI_API_KEY is not set in the environment variables")

In [ ]:
from agents import Agent, Runner
from pydantic import BaseModel

class Tutorial(BaseModel):
    outline: str
    tutorial: str

tutorial_generator = Agent(
    name="Tutorial Generator",
    handoff_description="Used for generating a tutorial based on an outline.",
    instructions=(
        "Given a programming topic and an outline, your job is to generate code snippets for each section of the outline."
        "Format the tutorial in Markdown using a mix of text for explanation and code snippets for examples."
        "Where it makes sense, include comments in the code snippets to further explain the code."
    ),
    output_type=Tutorial
)

outline_builder = Agent(
    name="Outline Builder",
    instructions=(
        "Given a particular programming topic, your job is to help come up with a tutorial. You will do that by crafting an outline."
        "After making the outline, hand it to the tutorial generator agent."
    ),
    handoffs=[tutorial_generator]
)

tutorial_response = await Runner.run(outline_builder, "Loops in Java")
print(tutorial_response.final_output)

outline='1. Introduction to Loops in Java\n2. The `for` Loop\n   - Syntax\n   - Example: Counting from 1 to 5\n3. The `while` Loop\n   - Syntax\n   - Example: Countdown from 5\n4. The `do-while` Loop\n   - Syntax\n   - Example: Print numbers until reaching 5\n5. Enhanced `for` Loop (for-each)\n   - Syntax\n   - Example: Iterating over an array\n6. Nested Loops\n   - Example: Multiplication table\n7. Best Practices\n   - Avoiding infinite loops\n   - Loop control statements' tutorial='# Loops in Java\n\nLoops are a fundamental part of Java programming that allow you to execute a block of code multiple times. Understanding loops will help you handle tasks that require repetition efficiently.\n\n## 1. The `for` Loop\n\n### Syntax\n\nThe `for` loop is used for iterating over a range. It consists of three parts:\n\n- Initialization\n- Condition\n- Update\n\n```java\nfor(initialization; condition; update) {\n    // Code to be executed\n}\n```\n\n### Example: Counting from 1 to 5\n\n```java\n

In [11]:
from agents import Agent, Runner, handoff, RunContextWrapper

history_tutor_agent = Agent(
    name="History Tutor",
    handoff_description="Specialist agent for historical questions",
    instructions="You provide assistance with historical queries. Explain important events and context clearly.",
)

math_tutor_agent = Agent(
    name="Math Tutor",
    handoff_description="Specialist agent for math questions",
    instructions="You provide assistance with math queries. Explain your reasoning at each step and include examples"
)

def on_math_handoff(ctx: RunContextWrapper[None]):
    print("Handing off to math tutor agent")

def on_history_handoff(ctx: RunContextWrapper[None]):
    print("Handing off to history tutor agent")

# This agent has the capability to handoff to either the history or math tutor agent
triage_agent = Agent(
    name="Triage Agent",
    instructions="You determine which agent to use based on the user's homework question." +
    "If neither agent is relevant, provide a general response.",
    handoffs=[handoff(history_tutor_agent, on_handoff=on_history_handoff), 
              handoff(math_tutor_agent, on_handoff=on_math_handoff)]
)

In [12]:
result = await Runner.run(triage_agent, "How do I add 2 and 2?")
result.final_output

Handing off to math tutor agent


'Adding 2 and 2 is a basic arithmetic operation. Here’s how you do it:\n\n1. **Understand the Operation**: Addition is the process of combining two or more numbers to get a total or sum.\n\n2. **Line Up the Numbers**: Since these are single-digit numbers, you can simply align them:\n   \\[\n   2 \\\\\n   + 2\n   \\]\n\n3. **Add the Numbers**: Start from the right (though here we only have one column):\n   \\[\n   2 + 2 = 4\n   \\]\n\n4. **Write Down the Sum**: The sum of 2 and 2 is 4.\n\n**Example**: If you have 2 apples and someone gives you 2 more apples, you count all the apples together to get 4 apples.\n\nThis method applies to all basic additions: align the numbers and add starting from the rightmost column.'

In [13]:
result = await Runner.run(triage_agent, "How did WW2 start?")
result.final_output

Handing off to history tutor agent


'World War II began on September 1, 1939, when Nazi Germany, led by Adolf Hitler, invaded Poland. This invasion prompted Britain and France to declare war on Germany two days later, on September 3, 1939.\n\n### Context Leading to the War:\n\n1. **Treaty of Versailles (1919):** After World War I, the Treaty of Versailles placed heavy reparations and territorial losses on Germany. This caused economic hardship and resentment among Germans.\n\n2. **Rise of Fascism:** During the 1920s and 1930s, fascist regimes emerged in several countries. Hitler\'s rise to power in Germany in 1933 was a significant turning point, as he sought to overturn the post-World War I order and expand German territory.\n\n3. **Expansion and Appeasement:**\n   - **Reoccupation of the Rhineland (1936):** Germany reoccupied this demilitarized zone, violating the Treaty of Versailles.\n   - **Annexation of Austria (1938):** Known as the Anschluss, Austria was annexed into Germany.\n   - **Munich Agreement (1938):** Br

In [15]:
from agents import function_tool

class ManagerEscalation(BaseModel):
    issue: str # the issue being escalated
    why: str # why can you not handle it? Used for training in the future

@function_tool
def create_ticket(issue: str):
    """"
    Create a ticket in the system for an issue to be resolved.
    """
    print(f"Creating ticket for issue: {issue}")
    return "Ticket created. ID: 12345"
    # In a real-world scenario, this would interact with a ticketing system

manager_agent = Agent(
    name="Manager",
    handoff_description="Handles escalated issues that require managerial attention",
    instructions=(
        "You handle escalated customer issues that the initial custom service agent could not resolve. "
        "You will receive the issue and the reason for escalation. If the issue cannot be immediately resolved for the "
        "customer, create a ticket in the system and inform the customer."
    ),
    tools=[create_ticket],
)

def on_manager_handoff(ctx: RunContextWrapper[None], input: ManagerEscalation):
    print("Escalating to manager agent: ", input.issue)
    print("Reason for escalation: ", input.why)

    # here we might store the escalation in a database or log it for future reference

customer_service_agent = Agent(
    name="Customer Service",
    instructions="You assist customers with general inquiries and basic troubleshooting. " +
                 "If the issue cannot be resolved, escalate it to the Manager along with the reason why you cannot fix the issue yourself.",
    handoffs=[handoff(
        agent=manager_agent,
        input_type=ManagerEscalation,
        on_handoff=on_manager_handoff,
    )]
)

result = await Runner.run(customer_service_agent, "Hello how much are tickets?")
print(result.final_output)

Could you please specify which event or destination you are inquiring about for ticket prices?


#### Recommended Prompt Prefix
The recommended prompt prefix is a constant that OpenAI recommends you give to handoff agents to improve understanding.

In [ ]:
from agents.extensions.handoff_prompt import RECOMMENDED_PROMPT_PREFIX

billing_agent = Agent(
    name="Billing agent",
    instructions=f"""{RECOMMENDED_PROMPT_PREFIX}
    <Fill in the rest of your prompt here>.""",
)

'# System context\nYou are part of a multi-agent system called the Agents SDK, designed to make agent coordination and execution easy. Agents uses two primary abstraction: **Agents** and **Handoffs**. An agent encompasses instructions and tools and can hand off a conversation to another agent when appropriate. Handoffs are achieved by calling a handoff function, generally named `transfer_to_<agent_name>`. Transfers between agents are handled seamlessly in the background; do not mention or draw attention to these transfers in your conversation with the user.\n'